# TAB ERGONOMICS

#### imports

In [55]:
import numpy as np
import itertools
import pandas as pd

## calculate fret change per string

In [69]:
frame_1 = [7,-1,3,-1,-1,3]
frame_2 = [10,-1,0,7,3,3]

In [70]:
def string_distance(curr,prev):
    if curr==prev: #if no string changed
        return 0 # zero distance
    if curr==-1 or curr==0 or prev==-1 or prev==0: 
    # this means that a string is moving from/to a non-played or non-fretted position
        return 1 # this is the smallest change   
    return abs(curr-prev)

In [194]:
def frame_distance(curr,prev):
    frame_changes = list(map(string_distance, curr, prev))
    total_change = np.sum(frame_changes)
    return total_change

In [71]:
frame_changes = list(map(string_distance,frame_1,frame_2))

In [72]:
frame_changes

[3, 0, 1, 1, 1, 0]

## total fret change is sum of fret change of every string

In [73]:
total_change = np.sum(frame_changes)
print(total_change)

6


## take frame and make it into multiple matrices for each note

In [44]:
pitch_matrix = [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57],
      [45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62],
      [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67],
      [55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72],
      [59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76],
      [64, 65, 66, 67, 68, 69, 70, 71, 72, 73 ,74, 75, 76, 77, 78, 79, 80, 81]]

In [ ]:
E
A
D 
G 
B 
e

In [50]:
def frame_to_midi(frame):
    midi = []
    for i in range(0,len(frame)): # for each string
        if frame[i] == -1:
            midi.append(-1)
        else:
            midi.append(pitch_matrix[i][frame[i]])
    return np.array(midi)

In [51]:
frame_to_midi(frame_1)

array([-1, -1, 53, -1, -1, 67])

In [74]:
frame_2

[10, -1, 0, 7, 3, 3]

In [52]:
midi = frame_to_midi(frame_2)

In [53]:
midi

array([50, -1, 50, 62, 62, 67])

In [96]:
np.unique(midi)

array([-1, 50, 62, 67])

## get all string/fret combinations for that frame

In [60]:
np.where(midi==50)

(array([0, 2]),)

In [165]:
midi = frame_to_midi(frame_2)
midi_notes = []
for note in np.unique(midi):
    note_frets=[]
    if note!=-1:
        for idx in np.where(midi == note)[0]:
            note_frets.append((idx, frame_2[idx]))
        midi_notes.append(note_frets)
print(midi_notes)

[[(0, 10), (2, 0)], [(3, 7), (4, 3)], [(5, 3)]]


In [169]:
combinations = list(itertools.product(*midi_notes))
combinations

[((0, 10), (3, 7), (5, 3)),
 ((0, 10), (4, 3), (5, 3)),
 ((2, 0), (3, 7), (5, 3)),
 ((2, 0), (4, 3), (5, 3))]

In [200]:
def combination_to_frames(combinations):
    result = []
    for combo in combinations:
        frame = {0:-1,1:-1,2:-1,3:-1,4:-1,5:-1}
        frame.update(dict(combo))
        result.append(list(frame.values()))
    return result

In [201]:
pd.DataFrame(combination_to_frames(combinations))

,0,1,2,3,4,5
0,10,-1,-1,7,-1,3
1,10,-1,-1,-1,3,3
2,-1,-1,0,7,-1,3
3,-1,-1,0,-1,3,3


In [202]:
options = combination_to_frames(combinations)

In [203]:
options

[[10, -1, -1, 7, -1, 3],
 [10, -1, -1, -1, 3, 3],
 [-1, -1, 0, 7, -1, 3],
 [-1, -1, 0, -1, 3, 3]]

In [204]:
frame_1 = [3,5,2,7,0,-1]

In [207]:
for option in options:
    print(frame_distance(frame_1,option))


11
12
5
6


In [193]:
def get_all_combinations(frame):
    #convert frame to midi notes
    midi = frame_to_midi(frame)
    midi_notes = []
    for note in np.unique(midi):
        note_frets=[]
        if note!=-1:
            for idx in np.where(midi == note)[0]:
                note_frets.append((idx, frame[idx]))
            midi_notes.append(note_frets)
    combinations = list(itertools.product(*midi_notes))
    return combination_to_frames(combinations)

[((0, 10), (3, 7), (5, 3)),
 ((0, 10), (4, 3), (5, 3)),
 ((2, 0), (3, 7), (5, 3)),
 ((2, 0), (4, 3), (5, 3))]

In [109]:
midi_bins

""


## calculate total fret change for all possibilities

In [208]:
for option in options:
    print(frame_distance(frame_1, option))


11
12
5
6


## for the possibilities with lowest fret change, calculate variance

In [316]:
frame_2

[10, -1, 0, 7, 3, 3]

In [317]:
frame_1

[3, 5, 2, 7, 0, -1]

In [318]:
options = get_all_combinations(frame_2)
options

[[10, -1, -1, 7, -1, 3],
 [10, -1, -1, -1, 3, 3],
 [-1, -1, 0, 7, -1, 3],
 [-1, -1, 0, -1, 3, 3]]

In [320]:
prev_frames = np.tile(frame_1,(len(options),1))

In [321]:
prev_frames

array([[ 3,  5,  2,  7,  0, -1],
       [ 3,  5,  2,  7,  0, -1],
       [ 3,  5,  2,  7,  0, -1],
       [ 3,  5,  2,  7,  0, -1]])

In [322]:
prev_frames

array([[ 3,  5,  2,  7,  0, -1],
       [ 3,  5,  2,  7,  0, -1],
       [ 3,  5,  2,  7,  0, -1],
       [ 3,  5,  2,  7,  0, -1]])

In [323]:
options

[[10, -1, -1, 7, -1, 3],
 [10, -1, -1, -1, 3, 3],
 [-1, -1, 0, 7, -1, 3],
 [-1, -1, 0, -1, 3, 3]]

In [324]:
options[3]=[-1,-1,2,-1,3,3]

In [325]:
options

[[10, -1, -1, 7, -1, 3],
 [10, -1, -1, -1, 3, 3],
 [-1, -1, 0, 7, -1, 3],
 [-1, -1, 2, -1, 3, 3]]

In [326]:
distance_matrix = np.array(list(map(frame_distance, options, prev_frames)))

In [327]:
distance_matrix

array([11, 12,  5,  5])

In [302]:
best_options_idx = list(np.where(distance_matrix == distance_matrix.min())[0])

In [307]:
best_options = []
for idx in best_options_idx:
    best_options.append(options[idx])

In [308]:
best_options

[[-1, -1, 0, 7, -1, 3], [-1, -1, 2, -1, 3, 3]]

In [309]:
variances = list(map(np.var, best_options))
variances

[8.805555555555555, 3.472222222222222]

In [310]:
best_options[np.argmin(variances)]

[-1, -1, 2, -1, 3, 3]

In [332]:
def best_frame(curr,prev):
    options = get_all_combinations(curr)
    prev_frames = np.tile(prev, (len(options), 1))
    distance_matrix = np.array(list(map(frame_distance, options, prev_frames)))
    best_options_idx = list(
        np.where(distance_matrix == distance_matrix.min())[0])
    best_options = []
    for idx in best_options_idx:
        best_options.append(options[idx])
    variances = list(map(np.var,best_options))
    lowest_var_option = best_options[np.argmin(variances)]
    return lowest_var_option

In [333]:
best_frame(frame_2,frame_1)

[-1, -1, 0, 7, -1, 3]

## choose the possibility with lowest variance

## 

## 